In [2]:
import numpy as np
from cities.utils.data_grabber import find_repo_root, MSADataGrabberCSV, DataGrabberCSV
import pandas as pd
import numpy as np
import requests
from us import states

root = find_repo_root()



In [6]:


# householdS1901_C01_013E mean income
# S1901_C01_012E median income 

SyntaxError: invalid syntax (2301551033.py, line 1)

In [7]:
# # code for retrieving available variables

variables_url = 'https://api.census.gov/data/2022/acs/acs5/subject/variables.json'


response = requests.get(variables_url)
variables_info = response.json()

variables_df = pd.DataFrame.from_dict(variables_info['variables'], orient='index').reset_index()

variables_df.to_excel(f'{root}/data/variables.xlsx', index=False)